In [1]:
import os
import gc
import warnings
import logging
import time
import math
import cv2
from pathlib import Path
import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from tqdm.auto import tqdm

# Suppress warnings and limit logging output
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
class CFG:
    """
    推論パイプラインに必要なすべてのパスとパラメータを保持する設定クラス。
    """
    # データパス
    test_soundscapes = 'test_soundscapes'  # テスト用の音響データのパス
    submission_csv = 'sample_submission.csv'  # 提出用CSVファイルのパス
    taxonomy_csv = 'taxonomy.csv'  # 種類情報のCSVファイルのパス
    model_path = '/kaggle/input/regnrty008/pytorch/default/1'  # モデルのパス
    
    # 音声データのパラメータ
    FS = 32000  # サンプリング周波数（32kHz）
    WINDOW_SIZE = 5  # ウィンドウサイズ（秒単位）
    
    # メルスペクトログラムのパラメータ
    N_FFT = 1024  # FFTサイズ
    HOP_LENGTH = 256
    N_MELS = 256
    FMIN = 50  # 最小周波数（Hz） #20
    FMAX = 14000  # 最大周波数（Hz）　#16000
    TARGET_SHAPE = (256, 256)  # ターゲット画像の形状
    
    # モデル関連
    model_name = 'regnety_008'  # 使用するモデル名
    in_channels = 1  # 入力チャンネル数（モノラル音声）
    device = 'cpu'  # 使用するデバイス（'cpu'または'cuda'）
    
    # 推論パラメータ
    use_tta = False  # TTA（Test Time Augmentation）を使用するかどうか
    tta_count = 3  # TTAの回数
    threshold = 0.7  # クラスの予測確信度閾値
    
    # モデル選択に関する設定
    use_specific_folds = False  # 特定のfoldを使用するかどうか
    folds = [0, 1]  # use_specific_foldsがTrueの場合に使用するfoldのリスト
    
    # デバッグ用設定
    debug = False  # デバッグモードのオンオフ
    debug_count = 5  # デバッグモード時に使用するサンプル数

    if debug:
        test_soundscapes = '/kaggle/input/birdclef-2025/train_soundscapes'  # テスト用の音響データのパス

In [ ]:
class BirdCLEF2025Pipeline:
    """
    BirdCLEF-2025の推論タスクのためのパイプライン。

    このクラスは、以下の一連の処理を整理します：
      - タクソノミーデータのロード。
      - トレーニングされたモデルのロードと準備。
      - オーディオファイルをメルスペクトログラムに変換。
      - 各オーディオセグメントの予測を実行。
      - 提出ファイルの作成。
      - 提出結果の後処理（予測の平滑化）。
    """

    class BirdCLEFModel(nn.Module):
        """
        BirdCLEF-2025のカスタムニューラルネットワークモデル（timmバックボーンを使用）。
        """
        def __init__(self, cfg, num_classes):
            """
            BirdCLEFModelの初期化。
            
            :param cfg: 設定パラメータ。
            :param num_classes: 出力クラスの数。
            """
            super().__init__()
            self.cfg = cfg
            # timmバックボーンを作成（指定されたパラメータを使用）
            self.backbone = timm.create_model(
                cfg.model_name,
                pretrained=False,  
                in_chans=cfg.in_channels,
                drop_rate=0.0,    
                drop_path_rate=0.0
            )
            # モデルのタイプに基づいて最終層を調整
            if 'efficientnet' in cfg.model_name:
                backbone_out = self.backbone.classifier.in_features
                self.backbone.classifier = nn.Identity()
            elif 'resnet' in cfg.model_name:
                backbone_out = self.backbone.fc.in_features
                self.backbone.fc = nn.Identity()
            else:
                backbone_out = self.backbone.get_classifier().in_features
                self.backbone.reset_classifier(0, '')
            
            self.pooling = nn.AdaptiveAvgPool2d(1)  # グローバル平均プーリング
            self.feat_dim = backbone_out
            self.classifier = nn.Linear(backbone_out, num_classes)  # クラス分類層
            
        def forward(self, x):
            """
            ネットワークの順伝播。
            
            :param x: 入力テンソル。
            :return: 各クラスのロジット。
            """
            features = self.backbone(x)  # バックボーンで特徴量を抽出
            if isinstance(features, dict):
                features = features['features']
            # 特徴量が4次元であれば、グローバル平均プーリングを適用
            if len(features.shape) == 4:
                features = self.pooling(features)
                features = features.view(features.size(0), -1)
            logits = self.classifier(features)  # 最後に分類層を適用
            return logits

    def __init__(self, cfg):
        """
        推論パイプラインの初期化。
        
        :param cfg: 設定オブジェクト（パスやパラメータを含む）。
        """
        self.cfg = cfg
        self.taxonomy_df = None
        self.species_ids = []
        self.models = []
        self._load_taxonomy()  # タクソノミーデータのロード

    def _load_taxonomy(self):
        """
        タクソノミーデータをCSVからロードし、種識別子を抽出。
        """
        print("タクソノミーデータをロードしています...")
        self.taxonomy_df = pd.read_csv(self.cfg.taxonomy_csv)
        self.species_ids = self.taxonomy_df['primary_label'].tolist()  # 主ラベル（種識別子）の抽出
        print(f"クラス数: {len(self.species_ids)}")

    def audio2melspec(self, audio_data):
        """
        生のオーディオデータを正規化したメルスペクトログラムに変換。
        
        :param audio_data: 1Dのnumpy配列としてのオーディオサンプル。
        :return: 正規化されたメルスペクトログラム。
        """
        if np.isnan(audio_data).any():
            mean_signal = np.nanmean(audio_data)
            audio_data = np.nan_to_num(audio_data, nan=mean_signal)
        
        mel_spec = librosa.feature.melspectrogram(
            y=audio_data,
            sr=self.cfg.FS,
            n_fft=self.cfg.N_FFT,
            hop_length=self.cfg.HOP_LENGTH,
            n_mels=self.cfg.N_MELS,
            fmin=self.cfg.FMIN,
            fmax=self.cfg.FMAX,
            power=2.0
        )
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8)
        return mel_spec_norm

    def process_audio_segment(self, audio_data):
        """
        オーディオセグメントを処理し、ターゲット形状のメルスペクトログラムを取得。
        
        :param audio_data: 1Dのnumpy配列としてのオーディオサンプル。
        :return: 処理されたメルスペクトログラム（float32のnumpy配列）。
        """
        # 必要なウィンドウサイズよりオーディオが短い場合はパディング
        if len(audio_data) < self.cfg.FS * self.cfg.WINDOW_SIZE:
            audio_data = np.pad(
                audio_data,
                (0, self.cfg.FS * self.cfg.WINDOW_SIZE - len(audio_data)),
                mode='constant'
            )
        
        mel_spec = self.audio2melspec(audio_data)  # メルスペクトログラムに変換
        
        # 必要であれば、ターゲット形状にリサイズ
        if mel_spec.shape != self.cfg.TARGET_SHAPE:
            mel_spec = cv2.resize(mel_spec, self.cfg.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR)
            
        return mel_spec.astype(np.float32)

    def find_model_files(self):
        """
        指定されたモデルディレクトリからすべての.pthモデルファイルを検索。
        
        :return: モデルファイルのパスリスト。
        """
        model_files = []
        model_dir = Path(self.cfg.model_path)
        for path in model_dir.glob('**/*.pth'):
            model_files.append(str(path))
        return model_files

    def load_models(self):
        """
        見つかったすべてのモデルファイルをロードし、アンサンブル推論の準備をする。
        
        :return: ロードされたPyTorchモデルのリスト。
        """
        self.models = []
        model_files = self.find_model_files()  # モデルファイルの検索
        if not model_files:
            print(f"警告: {self.cfg.model_path}にモデルファイルが見つかりません!")
            return self.models

        print(f"合計{len(model_files)}個のモデルファイルが見つかりました。")
        
        # 特定のフォールドを使用する場合は、モデルファイルをフィルタリング
        if self.cfg.use_specific_folds:
            filtered_files = []
            for fold in self.cfg.folds:
                fold_files = [f for f in model_files if f"fold{fold}" in f]
                filtered_files.extend(fold_files)
            model_files = filtered_files
            print(f"指定されたフォールド（{self.cfg.folds}）に対して{len(model_files)}個のモデルファイルを使用します。")
        
        # 各モデルファイルをロード
        for model_path in model_files:
            try:
                print(f"モデルをロード中: {model_path}")
                checkpoint = torch.load(model_path, map_location=torch.device(self.cfg.device))
                model = self.BirdCLEFModel(self.cfg, len(self.species_ids))
                model.load_state_dict(checkpoint['model_state_dict'])
                model = model.to(self.cfg.device)
                model.eval()  # 推論モード
                self.models.append(model)
            except Exception as e:
                print(f"モデル{model_path}のロード中にエラーが発生しました: {e}")
        
        return self.models

    def apply_tta(self, spec, tta_idx):
        """
        テスト時拡張（TTA）をメルスペクトログラムに適用。
        
        :param spec: 入力メルスペクトログラム。
        :param tta_idx: 適用するTTAのインデックス。
        :return: 拡張後のメルスペクトログラム。
        """
        if tta_idx == 0:
            # 拡張なし
            return spec
        elif tta_idx == 1:
            # 時間シフト（水平フリップ）
            return np.flip(spec, axis=1)
        elif tta_idx == 2:
            # 周波数シフト（垂直フリップ）
            return np.flip(spec, axis=0)
        else:
            return spec

    def predict_on_spectrogram(self, audio_path):
        """
        単一のオーディオファイルを処理し、各5秒セグメントに対して種の存在を予測。
        
        :param audio_path: オーディオファイルのパス。
        :return: 各セグメントに対する(row_id, predictions)のタプル。
        """
        predictions = []
        row_ids = []
        soundscape_id = Path(audio_path).stem
        
        try:
            print(f"{soundscape_id}を処理中...")
            audio_data, _ = librosa.load(audio_path, sr=self.cfg.FS)
            total_segments = int(len(audio_data) / (self.cfg.FS * self.cfg.WINDOW_SIZE))
            
            for segment_idx in range(total_segments):
                start_sample = segment_idx * self.cfg.FS * self.cfg.WINDOW_SIZE
                end_sample = start_sample + self.cfg.FS * self.cfg.WINDOW_SIZE
                segment_audio = audio_data[start_sample:end_sample]
                
                end_time_sec = (segment_idx + 1) * self.cfg.WINDOW_SIZE
                row_id = f"{soundscape_id}_{end_time_sec}"
                row_ids.append(row_id)

                # TTAが有効な場合
                if self.cfg.use_tta:
                    all_preds = []
                    for tta_idx in range(self.cfg.tta_count):
                        mel_spec = self.process_audio_segment(segment_audio)
                        mel_spec = self.apply_tta(mel_spec, tta_idx)
                        mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                        mel_spec_tensor = mel_spec_tensor.to(self.cfg.device)

                        if len(self.models) == 1:
                            with torch.no_grad():
                                outputs = self.models[0](mel_spec_tensor)
                                probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                                all_preds.append(probs)
                        else:
                            segment_preds = []
                            for model in self.models:
                                with torch.no_grad():
                                    outputs = model(mel_spec_tensor)
                                    probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                                    segment_preds.append(probs)
                            avg_preds = np.mean(segment_preds, axis=0)
                            all_preds.append(avg_preds)
                    final_preds = np.mean(all_preds, axis=0)
                else:
                    mel_spec = self.process_audio_segment(segment_audio)
                    mel_spec_tensor = torch.tensor(mel_spec, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
                    mel_spec_tensor = mel_spec_tensor.to(self.cfg.device)
                    
                    if len(self.models) == 1:
                        with torch.no_grad():
                            outputs = self.models[0](mel_spec_tensor)
                            final_preds = torch.sigmoid(outputs).cpu().numpy().squeeze()
                    else:
                        segment_preds = []
                        for model in self.models:
                            with torch.no_grad():
                                outputs = model(mel_spec_tensor)
                                probs = torch.sigmoid(outputs).cpu().numpy().squeeze()
                                segment_preds.append(probs)
                        final_preds = np.mean(segment_preds, axis=0)
                
                predictions.append(final_preds)
        except Exception as e:
            print(f"{audio_path}の処理中にエラーが発生しました: {e}")
        
        return row_ids, predictions

    def run_inference(self):
        """
        すべてのテスト音景オーディオファイルに対して推論を実行。
        
        :return: すべてのファイルから集計された(row_ids, predictions)のタプル。
        """
        test_files = list(Path(self.cfg.test_soundscapes).glob('*.ogg'))  # テスト音景ファイルを取得
        if self.cfg.debug:
            print(f"デバッグモードが有効です。{self.cfg.debug_count}個のファイルのみ使用")
            test_files = test_files[:self.cfg.debug_count]
        print(f"{len(test_files)}個のテスト音景ファイルが見つかりました")

        all_row_ids = []
        all_predictions = []

        for audio_path in tqdm(test_files):
            row_ids, predictions = self.predict_on_spectrogram(str(audio_path))
            all_row_ids.extend(row_ids)
            all_predictions.extend(predictions)
        
        return all_row_ids, all_predictions

    def create_submission(self, row_ids, predictions):
        """
        予測結果に基づいて提出用のデータフレームを作成。
        
        :param row_ids: 各セグメントの識別子のリスト。
        :param predictions: 予測結果のリスト。
        :return: 提出用フォーマットに整形されたpandas DataFrame。
        """
        print("提出用データフレームを作成中...")
        submission_dict = {'row_id': row_ids}
        for i, species in enumerate(self.species_ids):
            submission_dict[species] = [pred[i] for pred in predictions]

        submission_df = pd.DataFrame(submission_dict)
        submission_df.set_index('row_id', inplace=True)

        sample_sub = pd.read_csv(self.cfg.submission_csv, index_col='row_id')
        missing_cols = set(sample_sub.columns) - set(submission_df.columns)
        if missing_cols:
            print(f"警告: 提出結果に{len(missing_cols)}種類の種が欠落しています")
            for col in missing_cols:
                submission_df[col] = 0.0

        submission_df = submission_df[sample_sub.columns]  # 提出形式に整える
        submission_df = submission_df.reset_index()
        
        return submission_df

    def smooth_submission(self, submission_path):
        """
        提出CSVの予測結果を後処理し、時間的一貫性を保つように予測を平滑化。
        
        各音景（'row_id'の最後のアンダースコア前部分でグループ化）について、各行の予測を隣接する行と平均化します。
        
        :param submission_path: 提出CSVファイルのパス。
        """
        print("提出結果の予測を平滑化しています...")
        sub = pd.read_csv(submission_path)
        cols = sub.columns[1:]
        # 'row_id'を基にグループを抽出
        groups = sub['row_id'].str.rsplit('_', n=1).str[0].values
        unique_groups = np.unique(groups)
        
        for group in unique_groups:
            idx = np.where(groups == group)[0]
            sub_group = sub.iloc[idx].copy()
            predictions = sub_group[cols].values
            new_predictions = predictions.copy()
            
            if predictions.shape[0] > 1:
                # 隣接するセグメントとの予測を平均化して平滑化
                new_predictions[0] = (predictions[0] * 0.8) + (predictions[1] * 0.2)
                new_predictions[-1] = (predictions[-1] * 0.8) + (predictions[-2] * 0.2)
                for i in range(1, predictions.shape[0]-1):
                    new_predictions[i] = (predictions[i-1] * 0.2) + (predictions[i] * 0.6) + (predictions[i+1] * 0.2)
            sub.iloc[idx, 1:] = new_predictions
        
        sub.to_csv(submission_path, index=False)
        print(f"平滑化された提出結果を{submission_path}に保存しました")

    def run(self):
        """
        完全な推論パイプラインを実行するメインメソッド。
    
        このメソッドでは、以下の処理を行います：
          - 事前学習されたモデルをロード。
          - テスト音声ファイルを処理して予測を実行。
          - 提出用のCSVを作成。
          - 予測値にスムージングを適用。
        """
        start_time = time.time()
        print("BirdCLEF-2025 推論開始...")
        print(f"TTA有効: {self.cfg.use_tta} (変動数: {self.cfg.tta_count if self.cfg.use_tta else 0})")
    
        self.load_models()
        if not self.models:
            print("モデルが見つかりませんでした！モデルパスを確認してください。")
            return
    
        print(f"使用モデル: {'単一モデル' if len(self.models) == 1 else f'{len(self.models)}モデルのアンサンブル'}")
        row_ids, predictions = self.run_inference()
        submission_df = self.create_submission(row_ids, predictions)
    
        submission_path = 'submission.csv'
        submission_df.to_csv(submission_path, index=False)
        print(f"初期提出ファイルが {submission_path} に保存されました")
    
        # 提出結果にスムージングを適用
        self.smooth_submission(submission_path)
    
        end_time = time.time()
        print(f"推論完了 (所要時間: {(end_time - start_time) / 60:.2f} 分)")

In [ ]:
if __name__ == "__main__":
    cfg = CFG()
    print(f"Using device: {cfg.device}")
    pipeline = BirdCLEF2025Pipeline(cfg)
    pipeline.run()  # Use the correct method name here

Using device: cpu
タクソノミーデータをロードしています...
クラス数: 206
BirdCLEF-2025 推論開始...
TTA有効: False (変動数: 0)
合計5個のモデルファイルが見つかりました。
モデルをロード中: /kaggle/input/regnrty008/pytorch/default/1/model_fold0.pth
モデルをロード中: /kaggle/input/regnrty008/pytorch/default/1/model_fold3.pth
モデルをロード中: /kaggle/input/regnrty008/pytorch/default/1/model_fold1.pth
モデルをロード中: /kaggle/input/regnrty008/pytorch/default/1/model_fold2.pth
モデルをロード中: /kaggle/input/regnrty008/pytorch/default/1/model_fold4.pth
使用モデル: 5モデルのアンサンブル
0個のテスト音景ファイルが見つかりました


0it [00:00, ?it/s]

提出用データフレームを作成中...
初期提出ファイルが submission.csv に保存されました
提出結果の予測を平滑化しています...
平滑化された提出結果をsubmission.csvに保存しました
推論完了 (所要時間: 0.06 分)
